# Python: Аналитика и визуализация

**Pandas + Matplotlib + Seaborn** — стандарт для аналитиков.

## Содержание
1. Загрузка и осмотр данных  
2. Очистка и трансформация  
3. Группировка и агрегация  
4. Визуализация (профессиональный уровень)  
5. Полезные функции  

---

## 1. Загрузка и осмотр данных

```python
import pandas as pd

# Загрузка
df = pd.read_csv('sales.csv')  # Читает CSV в DataFrame

# Осмотр
df.head()           # Первые 5 строк — быстрый взгляд на данные
df.info()           # Типы колонок, объём памяти, пропуски
df.describe()       # Статистика по числовым колонкам (min, max, mean, quartiles)
df.isna().sum()     # Количество пропусков по каждой колонке
```

> **Зачем:** Понимаем структуру, типы, пропуски.

## 2. Очистка и трансформация

```python
# Пропуски
df.fillna(0)                        # Заменяет все NaN на 0
df.dropna(subset=['amount'])        # Удаляет строки, где amount = NaN

# Типы
df['date'] = pd.to_datetime(df['date'])                     # Преобразует строку в datetime
df['amount'] = pd.to_numeric(df['amount'], errors='coerce') # Преобразует в число, нечисловые → NaN

# Категоризация
bins = [0, 1000, 10000, float('inf')]     # Границы интервалов: от 0 до 1000, от 1000 до 10000, от 10000 и выше
labels = ['low', 'medium', 'high']        # Названия категорий для каждого интервала
df['tier'] = pd.cut(df['amount'], bins=bins, labels=labels)  # Разбивает amount на категории

# Что делает pd.cut:
# Делит числовой столбец amount на интервалы по заданным границам (bins) и присваивает метки (labels) каждой категории.
# Пример:
# amount = 500  → tier = 'low'
# amount = 5000 → tier = 'medium'
# amount = 20000 → tier = 'high'

# Фильтрация
df[df['возраст'] > 18]                          # Фильтр
df.query('возраст > 18 and город == "Москва"')  # SQL-подобный фильтр

```

> **Зачем:** Чистые данные — точные выводы.

## 3. Группировка и агрегация

```python
# Группировка
df.groupby('region').agg({
    'amount': ['sum', 'mean', 'count']   # Сумма, среднее, количество по регионам
})

# Pivot
df.pivot_table(
    values='amount', 
    index='region', 
    columns='month', 
    aggfunc='sum'                        # Таблица: строки — регионы, столбцы — месяцы, значения — сумма
)

# Топ
df.nlargest(5, 'amount')                 # 5 строк с наибольшим amount

# Для YoY (год к году)
# 1. Извлекаем год из даты
df['year'] = df['date'].dt.year  
# → Добавляем столбец 'year' (например, 2024, 2025)

# 2. Группируем: регион + год → сумма amount
yoy = df.groupby(['region', 'year'])['amount'].sum().unstack()
# → Получаем таблицу:
#        2024     2025
# Москва  1.2M    1.5M
# СПБ   0.8M    1.0M

# 3. Считаем % роста к предыдущему году
yoy['yoy_%'] = yoy.pct_change(axis=1).iloc[:, -1] * 100
# .pct_change(axis=1) → (2025 - 2024) / 2024
# .iloc[:, -1] → берём только последний столбец (2025 vs 2024)
# * 100 → в процентах

# Результат:
#        2024     2025   yoy_%
# Москва  1.2M    1.5M   25.0%
# СПБ   0.8M    1.0M   25.0%

```

> **Зачем:** Метрики по группам, YoY, план-факт.

## 4. Визуализации

```python
import seaborn as sns
import matplotlib.pyplot as plt
```


### Подготовка
```python

fig, ax = plt.subplots(figsize=(12, 6))  # Размер холста

# Стандартные размеры в ДЮЙМАХ:
figsize=(8, 6)     # Компактный
figsize=(10, 6)    # Стандартный  
figsize=(12, 8)    # Крупный (рекомендуемый)
figsize=(14, 10)   # Очень крупный
figsize=(16, 12)   # Презентационный

# Для нескольких графиков:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))   # 2 горизонтально
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))  # 2 вертикально

### Данные для графика
x = данные_для_оси_x
y = данные_для_оси_y

# Выбор типа графика:
ax.bar(x, y)                     # Столбчатая (вертикальная)
ax.barh(x, y)                    # Столбчатая (горизонтальная)
ax.plot(x, y)                    # Линейный
ax.scatter(x, y)                 # Точечная
ax.pie(sizes, labels=labels)     # Круговая
ax.hist(data, bins=20)           # Гистограмма
ax.colorbar(scatter)             # Цветовая панель

# Усиливаем различия для визуализации colorbar в дополнение к scatter
accuracy_sorted = line_accuracy.sort_values()
emphasized = (accuracy_sorted - 99.8) * 100

scatter = ax.scatter(emphasized, range(len(accuracy_sorted)),
                    s=200, c=accuracy_sorted, cmap='RdYlGn', alpha=0.7)

```

### Настройка внешнего вида
```python

# Обязательные настройки:
ax.set_title('Заголовок', fontsize=14, fontweight='bold', pad=20)
ax.set_xlabel('Ось X', fontsize=12)
ax.set_ylabel('Ось Y', fontsize=12)
ax.grid(True, alpha=0.3)                    # Сетка
ax.legend(fontsize=11)                      # Легенда
plt.tight_layout(pad=3.0)                   # Авто-отступы

# Для горизонтальных барчартов:
ax.barh(names, values, color='lightblue', edgecolor='navy', height=0.7)

# Стили графиков:
plt.style.use('seaborn-v0_8')    # Красивый (рекомендуемый)
plt.style.use('ggplot')          # Деловой
plt.style.use('bmh')             # Академический
plt.style.use('dark_background') # Темный

# Цветовые палитры:
sns.color_palette("husl")        # Яркие
sns.color_palette("pastel")      # Пастельные  
sns.color_palette("viridis")     # Scientific
plt.cm.tab10                     # Стандартные

# Подписи
# Автоматический отступ для текста на барах:
max_value = max(values)
offset = max_value * 0.03  # 3% от макс. значения

# Поворот подписей если длинные:
plt.xticks(rotation=45, ha='right')

# Ограничение осей:
ax.set_xlim(0, max_value * 1.1)  # +10% запаса

# Добавление значений на график:
for i, v in enumerate(values):
    ax.text(v + offset, i, f'{v}', va='center', fontweight='bold')

```

### Сохранение в файл:
```python
plt.savefig('график.png', dpi=300, bbox_inches='tight', facecolor='white')
# dpi=300          - Высокое качество
# bbox_inches='tight' - Обрезает пустые поля
# facecolor='white' - Белый фон
plt.show()
```

> **Зачем:** Читаемые дашборды для бизнеса. Подписи, стиль, экспорт.

## 5. Полезные функции

```python
# Применение — простые расчёты 
df['profit'] = df['revenue'] - df['cost']
# → Новый столбец: прибыль = выручка − затраты
df['margin'] = df.apply(lambda x: x['profit'] / x['revenue'] if x['revenue'] else 0, axis=1)
# → Маржа = прибыль / выручка (в долях)
# lambda x: ... — анонимная функция для каждой строки
# axis=1 — по строкам (axis=0 — по столбцам)
# if x['revenue'] else 0 — защита от деления на 0
# Пример: profit=400, revenue=1000 → margin=0.4

# Объединение
df_merged = pd.merge(df1, df2, left_on='id', right_on='user_id', how='left')
df_merged = df_merged.drop('user_id', axis=1)  # оставляем только 'id'

# Пример:
# df1:  id | revenue
#       1 | 1000
#       2 | 1500
#
# df2:  user_id | cost
#       1       | 600
#       3       | 800
#
# Результат:
#   id | revenue | cost
#    1 | 1000    | 600
#    2 | 1500    | NaN
```

> **Зачем:** Гибкость, автоматизация, как SQL JOIN.

## 6. Основные команды Folium (алгоритм создания карт)

```python
import folium
from folium.plugins import MarkerCluster, HeatMap
```

### Создание базовой карты 'OpenStreetMap'
```python
m = folium.Map(
    location=[55.7558, 37.6173],  # Центр карты [широта, долгота]
    zoom_start=10,                # Уровень приближения
    tiles='OpenStreetMap'         # Стиль карты
)

# Альтернативные стили карт:
# tiles='Stamen Terrain'    - Топографический
# tiles='Stamen Toner'      - Черно-белый
# tiles='CartoDB positron'  - Светлый
# tiles='CartoDB dark_matter' - Темный
```

### Добавление маркеров
```python
# Простой маркер
folium.Marker(
    location=[55.7558, 37.6173],
    popup='Текст при клике',
    tooltip='Текст при наведении',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

# Маркер с кастомной иконкой
folium.Marker(
    location=[55.7558, 37.6173],
    popup='<b>Жирный текст</b><br>Вторая строка',
    tooltip='Наведи на меня',
    icon=folium.Icon(
        color='green',          # Цвет: red, blue, green, purple, orange, etc.
        icon='home',           # Иконка: home, star, cloud, flag, etc.
        prefix='fa'            # Font Awesome иконки
    )
).add_to(m)
```

### Кластеризация маркеров (для многих точек)
```python
# Создаем кластер для группировки маркеров
marker_cluster = MarkerCluster().add_to(m)

# Добавляем много маркеров в кластер
for i in range(100):
    folium.Marker(
        location=[55.7558 + random.uniform(-0.5, 0.5), 
                  37.6173 + random.uniform(-0.5, 0.5)],
        popup=f'Маркер {i}',
        icon=folium.Icon(color='blue')
    ).add_to(marker_cluster)
```

### Тепловые карты (heatmap)
```python
# Данные для тепловой карты [широта, долгота, вес]
heat_data = [
    [55.7558, 37.6173, 1.0],
    [55.7517, 37.6178, 0.8],
    [55.7500, 37.6200, 0.6]
]

# Создаем тепловую карту
HeatMap(heat_data, min_opacity=0.2, radius=15, blur=10).add_to(m)
```

### Круги и полигоны
```python
# Круг на карте
folium.Circle(
    location=[55.7558, 37.6173],
    radius=500,                    # Радиус в метрах
    popup='Круг радиусом 500м',
    color='crimson',              # Цвет обводки
    fill=True,                    # Заливка
    fill_color='crimson',         # Цвет заливки
    fill_opacity=0.2              # Прозрачность заливки
).add_to(m)

# Прямоугольник
folium.Rectangle(
    bounds=[[55.75, 37.61], [55.76, 37.62]],  # [[юг, запад], [север, восток]]
    popup='Прямоугольник',
    color='blue',
    fill=True,
    fill_opacity=0.2
).add_to(m)
```

### POPUP с HTML-форматированием
```python
# Popup с красивым форматированием
html_popup = """
<h3>Заголовок</h3>
<b>Данные:</b> Значение<br>
<b>Данные:</b> Значение<br>
<b>Данные:</b> Значение
"""

folium.Marker(
    location=[55.7558, 37.6173],
    popup=folium.Popup(html_popup, max_width=300),  # Макс. ширина popup
    tooltip='Кликни для информации'
).add_to(m)
```

### Слои и группировка
```python
# Создаем слои для разных типов объектов
layer_1 = folium.FeatureGroup(name='Слой 1')
layer_2 = folium.FeatureGroup(name='Слой 2')

# Добавляем объекты в разные слои
folium.Marker([55.7558, 37.6173], popup='Слой 1').add_to(layer_1)
folium.Marker([55.7568, 37.6273], popup='Слой 1').add_to(layer_2)

# Добавляем слои на карту
layer_1.add_to(m)
layer_2.add_to(m)

# Добавляем контроль слоев
folium.LayerControl().add_to(m)
```

### СОХРАНЕНИЕ КАРТЫ
```python
# Сохраняем карту в HTML файл
m.save('моя_карта.html')

# Показываем карту в Jupyter
m
```

### Кастомные стили POPUP
```python
# Popup с CSS стилями
styled_popup = """
<div style="font-family: Arial; background-color: #f0f8ff; padding: 10px; border-radius: 5px;">
    <h4 style="color: #2c3e50; margin-bottom: 10px;"> {data1}</h4>
    <p><b> Данные:</b> {data2}</p>
    <p><b> Данные:</b> {data3}</p>
    <p><b> Данные:</b> {data4}</p>
</div>
""".format(
    data1="Текст",
    data2=123456,
    data3="Текст", 
    data4="Текст"
)

folium.Marker(
    location=[55.7558, 37.6173],
    popup=folium.Popup(styled_popup, max_width=300)
).add_to(m)
```

## 7. Универсальная функция для красивого отображения таблиц со Styler

```python
from IPython.display import display

def styled_table(df, title="", wrap_columns=None, max_width='300px'):
    """
    Универсальная функция для красивого отображения таблиц со Styler
    
    Parameters:
    -----------
    df : DataFrame
        Таблица для отображения
    title : str, optional
        Заголовок таблицы. Если не указан, заголовок не выводится
    wrap_columns : list, optional
        Список колонок для переноса текста. Если None, перенос применяется 
        ко всем текстовым колонкам (object и string типы)
    max_width : str, optional
        Максимальная ширина ячеек в CSS-формате (по умолчанию '300px')
    
    Examples:
    ---------
    >>> styled_table(df, "Моя таблица")
    >>> styled_table(df, "Отчет", wrap_columns=['Описание', 'Комментарий'])
    >>> styled_table(df, max_width='400px')
    """
    if title:
        print(f"\n{title}")
    
    styler = df.style
    
    # Определяем колонки для переноса (все текстовые если не указано)
    if wrap_columns is None:
        wrap_columns = df.select_dtypes(include=['object', 'string']).columns.tolist()
    
    # Применяем перенос текста
    properties = {
        'white-space': 'pre-wrap',    # Сохраняет пробелы и переносы
        'word-wrap': 'break-word',    # Разрешает перенос слов
        'max-width': max_width,       # Ограничивает ширину ячеек
        'text-align': 'left'          # Выравнивание по левому краю
    }
    
    for col in wrap_columns:
        if col in df.columns:
            styler = styler.set_properties(subset=[col], **properties)
    
    display(styler)
```
> **Зачем:** Красивое отображение таблиц в jupiter notebook, с корректным переносом текста.

### Примеры использования в других частях ноутбука

```python
# Простой вариант
styled_table(df, "Моя таблица")

# С указанием конкретных колонок для переноса
styled_table(
    df, 
    "Отчет по продажам",
    wrap_columns=['Описание', 'Комментарий'],
    number_columns=['Цена', 'Количество']
)

# С изменением ширины
styled_table(
    df,
    "Широкая таблица", 
    max_width='500px'
)
```

---
**Источники:** Pandas docs, Seaborn gallery.  
*Добавляйте свои графики через PR.*